Import Library

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer
from tfx.proto import example_gen_pb2
from tfx.proto import trainer_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import tensorflow_model_analysis as tfma
from tfx.components import Evaluator
import os

SET Variable


In [2]:
# Menyiapkan konteks interaktif
PIPELINE_NAME = "reynaldoabt-pipeline"

OUTPUT_BASE = 'outputs'
SERVING_MODEL_DIR = os.path.join(OUTPUT_BASE, 'serving_model')
PIPELINE_ROOT = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'metadata.sqlite')

interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [3]:
# Data Ingestion
DATA_ROOT = 'data'
example_gen = CsvExampleGen(input_base=DATA_ROOT)
interactive_context.run(example_gen)

# Data Validation
statistic_gen = StatisticsGen(examples=example_gen.outputs["examples"])
interactive_context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [4]:
schema_gen = SchemaGen(statistics=statistic_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [5]:
example_validator = ExampleValidator(
    statistics=statistic_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

# Data Preprocessing


In [6]:
TRANSFORM_MODULE_FILE = "transform.py"

In [7]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: outputs\reynaldoabt-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\8f97d5e812e546f39b9c3aa4c047677f\assets


INFO:tensorflow:Assets written to: outputs\reynaldoabt-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\8f97d5e812e546f39b9c3aa4c047677f\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [8]:
TRAINER_MODULE_FILE = "trainer.py"

In [9]:
# Menjalankan Trainer untuk melatih model
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=1000),  
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)       
)
interactive_context.run(trainer)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 gender_xf (InputLayer)         [(None, 3)]          0           []                               
                                                                                                  
 smoking_history_xf (InputLayer  [(None, 4)]         0           []                               
 )                                                                                                
                                                                                                  
 age_xf (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 hypertension_xf (InputLayer)   [(None, 1)]          0           []                           

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: outputs\reynaldoabt-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\reynaldoabt-pipeline\Trainer\model\6\Format-Serving\assets


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [10]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model = Channel(type = Model),
    model_blessing = Channel(type = ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [11]:
import tensorflow_transform as tft  

# Ambil path ke transform_graph  
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri  
print("Transform graph path:", transform_graph_uri)  

# Inisialisasi TFTransformOutput  
tf_transform_output = tft.TFTransformOutput(transform_graph_uri)  

Transform graph path: outputs\reynaldoabt-pipeline\Transform\transform_graph\5


In [14]:
import tensorflow as tf  
import tensorflow_transform as tft  
from tfx.components import Evaluator  
import tensorflow_model_analysis as tfma  

# Evaluator component  
eval_config = tfma.EvalConfig(  
    model_specs=[tfma.ModelSpec(label_key='diabetes')],  
    slicing_specs=[tfma.SlicingSpec()],  
    metrics_specs=[  
        tfma.MetricsSpec(metrics=[  
            tfma.MetricConfig(class_name='ExampleCount'),  
            tfma.MetricConfig(class_name='AUC'),  
            tfma.MetricConfig(class_name='FalsePositives'),  
            tfma.MetricConfig(class_name='TruePositives'),  
            tfma.MetricConfig(class_name='FalseNegatives'),  
            tfma.MetricConfig(class_name='TrueNegatives'),  
            tfma.MetricConfig(  
                class_name='BinaryAccuracy',  
                threshold=tfma.MetricThreshold(  
                    value_threshold=tfma.GenericValueThreshold(  
                        lower_bound={'value': 0.5}  
                    ),  
                    change_threshold=tfma.GenericChangeThreshold(  
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,  
                        absolute={'value': 0.0001}  
                    )  
                )  
            ),  
            tfma.MetricConfig(class_name='Precision'),  
            tfma.MetricConfig(class_name='Recall'),  
            tfma.MetricConfig(class_name='F1Score')  
        ])  
    ]  
)  

# Ambil path ke transform_graph dari output Transform  
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri  
print("Transform graph path:", transform_graph_uri)  

# Inisialisasi TFTransformOutput  
tf_transform_output = tft.TFTransformOutput(transform_graph_uri)  

# Inisialisasi tft_layer dalam model  
model = tf.keras.models.load_model(trainer.outputs['model'].get()[0].uri)  
model.tft_layer = tf_transform_output.transform_features_layer()  

# Evaluator  
evaluator = Evaluator(  
    examples=example_gen.outputs['examples'],  
    model=trainer.outputs['model'],  
    baseline_model=model_resolver.outputs['model'],  
    eval_config=eval_config  
)  

# Jalankan Evaluator  
interactive_context.run(evaluator)  

Transform graph path: outputs\reynaldoabt-pipeline\Transform\transform_graph\5


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [15]:
evaluator_output_uri = evaluator.outputs['evaluation'].get()[0].uri  
print("Evaluator output path:", evaluator_output_uri)  

Evaluator output path: outputs\reynaldoabt-pipeline\Evaluator\evaluation\8


In [19]:
import tensorflow_model_analysis as tfma  

# Path ke output Evaluator  
evaluator_output_uri = "outputs/reynaldoabt-pipeline/Evaluator/evaluation/8"  

# Cek path output  
print("Using Evaluator output path:", evaluator_output_uri)  

# Baca hasil evaluasi  
try:  
    results = tfma.load_eval_result(evaluator_output_uri)  
    print("Evaluation results loaded successfully.")  
except Exception as e:  
    print("Error loading evaluation results:", e)  

# Tampilkan struktur hasil  
print("Results structure:", results)  

# Render metrik  
try:  
    tfma.view.render_slicing_metrics(results)  
except Exception as e:  
    print("Error rendering metrics:", e)  

Using Evaluator output path: outputs/reynaldoabt-pipeline/Evaluator/evaluation/8
Evaluation results loaded successfully.
Results structure: EvalResult(slicing_metrics=[((), {'': {'': {'binary_accuracy': {'doubleValue': 0.9701326769526325}, 'loss': {'doubleValue': 0.0845356285572052}, 'example_count': {'doubleValue': 33314.0}, 'auc': {'doubleValue': 0.9772682659169101}, 'false_positives': {'doubleValue': 7.0}, 'true_positives': {'doubleValue': 1845.0}, 'false_negatives': {'doubleValue': 988.0}, 'true_negatives': {'doubleValue': 30474.0}, 'precision': {'doubleValue': 0.9962203023758099}, 'recall': {'doubleValue': 0.6512530885986587}, 'f1_score': {'doubleValue': 0.7876200640341515}}}})], plots=[((), None)], attributions=[((), None)], config=model_specs {
  label_key: "diabetes"
}
slicing_specs {
}
metrics_specs {
  metrics {
    class_name: "ExampleCount"
  }
  metrics {
    class_name: "AUC"
  }
  metrics {
    class_name: "FalsePositives"
  }
  metrics {
    class_name: "TruePositives"


In [ ]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model = trainer.outputs['model'],
    model_blessing = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory = "outputs/serving_model/"
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))